This script uses the quantile normalized data as well as the clinical and demographic data to build linear regression models between Active GCA patients and Healthy controls. Marginal linear regression models were run on six clinical and demographic variables (age, smoking status, sex, prednisone use, aspirin use, and methotrexate use) between Active GCA and Healthy controls. Variables with linear regression model P-values < 0.05 were identified as significant confounders and included in the full multiple linear regression models. In the full multiple linear regression models, the P-value from the study group variable was used to identify differentially abundant proteins between Active GCA and Healthy controls. A threshold of P < 0.01 was applied to all plasma proteins for significance.

In [1]:
#Protein ~ Study_group	Age	Smoking	Prednisone	Methotrexate	Aspirin
#function_name <- function(arg_1, arg_2, ...) {
#   Function body 
#}
make_linear_model <- function(binary_results){
    #binary_results is the row from the binary data, 7 columns with 6 values
    model_string = "~ Study_group"
    if(binary_results[1,2] == 1){
        model_string = paste0(model_string," + Age")
    }
    if(binary_results[1,3] == 1){
        model_string = paste0(model_string," + Sex")
    }
    if(binary_results[1,4] == 1){
        model_string = paste0(model_string," + Smoking")
    }
    if(binary_results[1,5] == 1){
        model_string = paste0(model_string," + Prednisone")
    }    
    if(binary_results[1,6] == 1){
        model_string = paste0(model_string," + Methotrexate")
    }    
    if(binary_results[1,7] == 1){
        model_string = paste0(model_string," + Aspirin")
    }    
    return(model_string)
}


In [12]:
library("dplyr")
library("ggpubr")
library("lme4")
library("lmerTest")
library("ggplot2")
#read in the proteomics data and the patient demographics data
#vasculitis_df <- read.csv("../../data/vasculitis_patients_final.csv")
vasculitis_demographics_df <- read.csv("../../data/vasculitis_patient_info.csv")

#read in all quantile data
all_quantile_df = read.csv("../../data/all_quantile_data.csv")
#head(all_quantile_df)
#get the number of columns and rows 
num_columns <- ncol(all_quantile_df)
num_proteins <- ncol(all_quantile_df) - 3
num_samples <- nrow(all_quantile_df)


#filter only active gca patients
active_df <- filter(all_quantile_df,all_quantile_df$Study_group == "Active")
active_demographics_df <- filter(vasculitis_demographics_df,vasculitis_demographics_df$Study_group == "Active")
active_proteins_only <- active_df[,4:num_columns]

#filter inactive gca patients
inactive_df = filter(all_quantile_df,all_quantile_df$Study_group == "Inactive")
inactive_demographics_df <- filter(vasculitis_demographics_df,vasculitis_demographics_df$Study_group == "Inactive")
inactive_proteins_only <- inactive_df[,4:num_columns]

#read in controls data
controls_demographics_df <- read.csv("../../data/controls_demographics.csv")
controls_df <- filter(all_quantile_df,all_quantile_df$Study_group == "Healthy_Control")
controls_proteins_only <- controls_df[,4:num_columns]

proteins_only <- data.frame(active_df[ ,4:num_columns])
#seperate the protein names for the linear model
protein_names <- data.frame(matrix(data=0,nrow=num_proteins,ncol=1))
colnames(protein_names) <- "Protein"
protein_names[ , 1] <- data.frame(colnames(proteins_only))
#protein_names



In [13]:
#gather the demographics I need for both active GCA and controls
#age,sex,smoking,prednisone,methotrexate,and aspirin
#check that the demographics and protein data is in the same order
print(active_demographics_df$maskid == active_df$maskID)
#head(active_demographics_df)
active_demographics_df = active_demographics_df[order(active_demographics_df$maskid),]
#head(active_demographics_df)
active_df = active_df[order(active_df$maskID),]
print(active_demographics_df$maskid == active_df$maskID)
active_gca_demographics_and_proteins_df <- data.frame(active_demographics_df$Age)
colnames(active_gca_demographics_and_proteins_df) <- ("Age")
active_gca_demographics_and_proteins_df$Sex <- active_demographics_df$Sex
active_gca_demographics_and_proteins_df$Smoking <- active_demographics_df$Smoking
active_gca_demographics_and_proteins_df$Prednisone <- active_demographics_df$PrednisoneCurrentlyReceiving
active_gca_demographics_and_proteins_df$Methotrexate <- active_demographics_df$MethotrexateWasTaken
active_gca_demographics_and_proteins_df$Aspirin <- active_demographics_df$Aspirin
active_gca_demographics_and_proteins_df2 <- cbind(active_gca_demographics_and_proteins_df,active_df)

dim(active_gca_demographics_and_proteins_df2)
#now I have all the active demographics and protein abundance together, do the same for controls

 [1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE


[1]   30 7298

In [14]:
print(controls_demographics_df$case_id == controls_df$maskID)
#put the controls data in order using case id for the demographics 
#and the maskID for the protein abundance
controls_demographics_df <- controls_demographics_df[order(controls_demographics_df$case_id),]
controls_df <- controls_df[order(controls_df$maskID),]
print(controls_demographics_df$case_id == controls_df$maskID)
#everything is in the same order, I can begin gather all demographic
#data that I need and combine with the protein abundance
control_demographics_and_proteins_df <- data.frame(controls_demographics_df$Age)
colnames(control_demographics_and_proteins_df) <- ("Age")
control_demographics_and_proteins_df$Sex <- controls_demographics_df$control_gender
control_demographics_and_proteins_df$Smoking <- controls_demographics_df$Smoking
control_demographics_and_proteins_df$Prednisone <- controls_demographics_df$Prednisone
control_demographics_and_proteins_df$Methotrexate <- controls_demographics_df$Methotrexate
control_demographics_and_proteins_df$Aspirin <- controls_demographics_df$Aspirin
control_demographics_and_proteins_df2 <- cbind(control_demographics_and_proteins_df,controls_df)
dim(control_demographics_and_proteins_df2)

 [1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
 [1] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE
[16] TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE TRUE


[1]   30 7298

In [15]:
#now combine everything and run linear models
active_and_controls_demographics_and_proteins <- rbind(active_gca_demographics_and_proteins_df2,control_demographics_and_proteins_df2)
dim(active_and_controls_demographics_and_proteins)
#active_and_controls_demographics_and_proteins

[1]   60 7298

In [16]:
#now I can start the linear modeling
#make a dataframe to hold the results
linear_modeling_results <- data.frame(matrix(data=0,nrow=num_proteins,ncol=7))
colnames(linear_modeling_results) <- cbind("Proteins","Age","Sex","Smoking","Prednisone","Methotrexate","Aspirin")
linear_modeling_results[,1] <- protein_names
head(linear_modeling_results)

,Proteins,Age,Sex,Smoking,Prednisone,Methotrexate,Aspirin
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CRBB2_10000.28,0,0,0,0,0,0
2,c.Raf_10001.7,0,0,0,0,0,0
3,ZNF41_10003.15,0,0,0,0,0,0
4,ELK1_10006.25,0,0,0,0,0,0
5,GUC1A_10008.43,0,0,0,0,0,0
6,BECN1_10010.10,0,0,0,0,0,0


In [17]:
#loop through all proteins using all covariates
for(i in 1:num_proteins){
    protein_name <- protein_names[i,1]
    age_model <- lm(paste0(protein_name, " ~ Age"),data = active_and_controls_demographics_and_proteins)
    pvalue1 <- summary(age_model)$coefficients[2,4]
    linear_modeling_results[i,2] <- pvalue1
    
    sex_model <- lm(paste0(protein_name, " ~ Sex"),data = active_and_controls_demographics_and_proteins)
    pvalue2 <- summary(sex_model)$coefficients[2,4]
    linear_modeling_results[i,3] <- pvalue2
    
    smoking_model <- lm(paste0(protein_name, " ~ Smoking"),data = active_and_controls_demographics_and_proteins)
    pvalue3 <- summary(smoking_model)$coefficients[2,4]
    linear_modeling_results[i,4] <- pvalue3
    
    prednisone_model <- lm(paste0(protein_name, " ~ Prednisone"),data = active_and_controls_demographics_and_proteins)
    pvalue4 <- summary(prednisone_model)$coefficients[2,4]
    linear_modeling_results[i,5] <- pvalue4
    
    methotrexate_model <- lm(paste0(protein_name, " ~ Methotrexate"),data = active_and_controls_demographics_and_proteins)
    pvalue5 <- summary(methotrexate_model)$coefficients[2,4]
    linear_modeling_results[i,6] <- pvalue5
    
    
    aspirin_model <- lm(paste0(protein_name, " ~ Aspirin"),data = active_and_controls_demographics_and_proteins)
    pvalue6 <- summary(aspirin_model)$coefficients[2,4]
    linear_modeling_results[i,7] <- pvalue6
    
}
print("Done with linear models.")

[1] "Done with linear models."


In [18]:
head(linear_modeling_results)

,Proteins,Age,Sex,Smoking,Prednisone,Methotrexate,Aspirin
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CRBB2_10000.28,0.29480239,0.6461081,0.4252281,1.340039e-01,0.78133572,0.33620378
2,c.Raf_10001.7,0.28663099,0.6079514,0.4346930,6.509082e-05,0.08232378,0.01140049
3,ZNF41_10003.15,0.39774211,0.6850563,0.5778376,9.775335e-02,0.02007203,0.87270196
4,ELK1_10006.25,0.24114861,0.1640577,0.9037085,6.154775e-01,0.56825094,0.94309655
5,GUC1A_10008.43,0.24964574,0.4586104,0.8429239,1.362117e-01,0.75566409,0.15702743
6,BECN1_10010.10,0.01456458,0.9462176,0.6055802,4.392471e-03,0.48779559,0.25928617


In [84]:
#save output
write.csv(linear_modeling_results,"../../analysis/linear_modeling_active_and_controls/Active_vs_Control_demographic_variables_linear_modeling_results.csv",row.names=FALSE)

In [19]:
#binarize the results
binary_modeling_results <- data.frame(matrix(data = 0,nrow=num_proteins,ncol=7))
colnames(binary_modeling_results) <- colnames(linear_modeling_results)
binary_modeling_results[,1] <- protein_names
for(i in 1:num_proteins){
    for(j in 2:7){
        current_pval <- linear_modeling_results[i,j]
        if(current_pval < 0.05){
            binary_modeling_results[i,j] <- 1
        }
        else{
            binary_modeling_results[i,j] <- 0
        }
    }    
}


In [86]:
#save binary results
write.csv(binary_modeling_results,"../../analysis/linear_modeling_active_and_controls/binary_modeling_results.csv",row.names=FALSE)

In [20]:
#I have a function that takes the binaryized data and returns a string to use in the model. Run for all proteins
adjusted_linear_modeling_results <- data.frame(matrix(data=0,nrow=num_proteins,ncol= 6))
colnames(adjusted_linear_modeling_results) <- cbind("Protein","Adjusted_linear_model","linear_modeling_pvalue","adjusted_pvalues","fold_change","coefficient")
adjusted_linear_modeling_results[,1] <- protein_names
for(i in 1:num_proteins){
    #get protein name
    current_protein <- protein_names[i,1]
    #make linear model string
    binary_results <- binary_modeling_results[i,]
    model_string <- ""
    model_string <- make_linear_model(binary_results)
    adjusted_linear_modeling_results[i,2] <- model_string
    adjusted_linear_model <- lm(paste0(current_protein,model_string),data = active_and_controls_demographics_and_proteins)
    pvalue1 <- summary(adjusted_linear_model)$coefficients[2,4]
    adjusted_linear_modeling_results[i,3] <- pvalue1
    coefficient_value <- summary(adjusted_linear_model)$coefficients[2,1]
    adjusted_linear_modeling_results[i,6] <- coefficient_value
}
print("Done with models.")

[1] "Done with models."


In [21]:
head(adjusted_linear_modeling_results)

,Protein,Adjusted_linear_model,linear_modeling_pvalue,adjusted_pvalues,fold_change,coefficient
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,CRBB2_10000.28,~ Study_group,0.006915540,0,0,-110.2248148
2,c.Raf_10001.7,~ Study_group + Prednisone + Aspirin,0.004575018,0,0,290.7460509
3,ZNF41_10003.15,~ Study_group + Methotrexate,0.976798964,0,0,0.1828778
4,ELK1_10006.25,~ Study_group,0.239878188,0,0,-177.7009815
5,GUC1A_10008.43,~ Study_group,0.231606319,0,0,-109.3417778
6,BECN1_10010.10,~ Study_group + Age + Prednisone,0.261359818,0,0,-27.4409527


In [39]:
#add the (BH) adjusted pvalues
adjusted_linear_modeling_results[,4] <- p.adjust(adjusted_linear_modeling_results$linear_modeling_pvalue, method="hochberg")


In [40]:
#find the means of active and controls
fold_changes <- data.frame(matrix(data=0,nrow=num_proteins,ncol=4))
colnames(fold_changes) <- cbind("Proteins", "Active_mean","Control_mean","fold_change")
fold_changes[,1] <- protein_names
for(i in 1:num_proteins){
    fold_changes[i,2] <- mean(active_proteins_only[,i])
    fold_changes[i,3] <- mean(controls_proteins_only[,i])
}
fold_changes[,4] <- log2(fold_changes[,2]/fold_changes[,3])

In [42]:
#add the fold changes
adjusted_linear_modeling_results[,5] <- fold_changes[,4]

#add entrez and target names
protein_key <- read.csv("../../data/key_for_protein_names.csv")
adjusted_linear_modeling_results$Target <- protein_key[,2]
adjusted_linear_modeling_results$Entrez <- protein_key[,3]
adjusted_linear_modeling_results$Uniprot = protein_key$Uniprot


#save the results
write.csv(adjusted_linear_modeling_results,"../../analysis/linear_modeling_active_and_controls/adjusted_linear_modeling_results.csv",row.names=FALSE)

In [43]:
#find the significant pvalues and summarize the models
significant_adjusted_linear_modeling_results <- filter(adjusted_linear_modeling_results,adjusted_linear_modeling_results$linear_modeling_pvalue < 0.01)
dim(significant_adjusted_linear_modeling_results)
significant_adjusted_linear_modeling_results <- significant_adjusted_linear_modeling_results[order(significant_adjusted_linear_modeling_results$linear_modeling_pvalue),]

#table(significant_adjusted_linear_modeling_results$Adjusted_linear_model)

[1] 537   9

In [27]:
#find those that are lower in active compared to controls
#and those that are higher in active compared to controls
higher_in_active_proteins <- filter(significant_adjusted_linear_modeling_results,significant_adjusted_linear_modeling_results$fold_change > 0)
dim(higher_in_active_proteins)
write.csv(higher_in_active_proteins,"../../analysis/linear_modeling_active_and_controls/higher_in_active_proteins.csv",row.names=FALSE)

lower_in_active_proteins <- filter(significant_adjusted_linear_modeling_results,significant_adjusted_linear_modeling_results$fold_change < 0)
dim(lower_in_active_proteins)
write.csv(lower_in_active_proteins,"../../analysis/linear_modeling_active_and_controls/lower_in_active_proteins.csv",row.names=FALSE)


[1] 202   9

[1] 335   9